# MLflow를 활용한 실험 추적

이 노트북은 MLflow를 사용하여 머신러닝 실험을 추적하고 관리하는 방법을 보여줍니다.

## 학습 내용
- MLflow 설정
- 실험 생성 및 실행
- 하이퍼파라미터 및 메트릭 로깅
- 모델 저장 및 관리
- 실험 비교

In [ ]:
# 필요한 라이브러리 설치
!pip install -q mlflow scikit-learn pandas numpy matplotlib

In [ ]:
# 라이브러리 임포트
import mlflow
import mlflow.sklearn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import warnings
warnings.filterwarnings('ignore')

print(f"MLflow 버전: {mlflow.__version__}")

## 1. 데이터 준비

In [ ]:
# Wine 데이터셋 로드
wine = load_wine()
X = wine.data
y = wine.target

# 학습/테스트 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# 스케일링
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"학습 데이터: {X_train_scaled.shape}")
print(f"테스트 데이터: {X_test_scaled.shape}")

## 2. MLflow 실험 설정

In [ ]:
# 실험 이름 설정
experiment_name = "wine-classification"
mlflow.set_experiment(experiment_name)

print(f"실험 이름: {experiment_name}")

## 3. 단일 실험 실행

In [ ]:
# Random Forest로 단일 실험 실행
with mlflow.start_run(run_name="rf-baseline"):
    # 파라미터 설정
    params = {
        'n_estimators': 100,
        'max_depth': 10,
        'random_state': 42
    }
    
    # 파라미터 로깅
    mlflow.log_params(params)
    
    # 모델 학습
    model = RandomForestClassifier(**params)
    model.fit(X_train_scaled, y_train)
    
    # 예측
    y_pred = model.predict(X_test_scaled)
    
    # 메트릭 계산 및 로깅
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    
    mlflow.log_metric("accuracy", accuracy)
    mlflow.log_metric("f1_score", f1)
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("recall", recall)
    
    # 모델 저장
    mlflow.sklearn.log_model(model, "model")
    
    print(f"Accuracy: {accuracy:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")

## 4. 여러 모델 비교 실험

In [ ]:
# 여러 모델 정의
models = {
    'RandomForest': RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42),
    'GradientBoosting': GradientBoostingClassifier(n_estimators=100, max_depth=5, random_state=42),
    'LogisticRegression': LogisticRegression(max_iter=1000, random_state=42)
}

# 각 모델에 대해 실험 실행
results = []

for model_name, model in models.items():
    with mlflow.start_run(run_name=f"{model_name.lower()}"):
        # 모델 이름 로깅
        mlflow.log_param("model_type", model_name)
        
        # 모델 파라미터 로깅
        for param, value in model.get_params().items():
            mlflow.log_param(param, value)
        
        # 모델 학습
        model.fit(X_train_scaled, y_train)
        
        # 예측
        y_pred = model.predict(X_test_scaled)
        
        # 메트릭 계산
        accuracy = accuracy_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred, average='weighted')
        precision = precision_score(y_test, y_pred, average='weighted')
        recall = recall_score(y_test, y_pred, average='weighted')
        
        # 메트릭 로깅
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("f1_score", f1)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        
        # 모델 저장
        mlflow.sklearn.log_model(model, "model")
        
        # 결과 저장
        results.append({
            'model': model_name,
            'accuracy': accuracy,
            'f1_score': f1,
            'precision': precision,
            'recall': recall
        })
        
        print(f"{model_name} - Accuracy: {accuracy:.4f}, F1: {f1:.4f}")

# 결과를 데이터프레임으로 변환
results_df = pd.DataFrame(results)
print("\n모델 비교 결과:")
print(results_df)

## 5. 하이퍼파라미터 튜닝 실험

In [ ]:
# Random Forest의 다양한 하이퍼파라미터 조합 시도
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [5, 10, 15]
}

tuning_results = []

for n_est in param_grid['n_estimators']:
    for max_d in param_grid['max_depth']:
        with mlflow.start_run(run_name=f"rf-tune-n{n_est}-d{max_d}"):
            # 파라미터 로깅
            mlflow.log_param("n_estimators", n_est)
            mlflow.log_param("max_depth", max_d)
            mlflow.log_param("model_type", "RandomForest")
            
            # 모델 학습
            model = RandomForestClassifier(
                n_estimators=n_est,
                max_depth=max_d,
                random_state=42
            )
            model.fit(X_train_scaled, y_train)
            
            # 예측 및 평가
            y_pred = model.predict(X_test_scaled)
            accuracy = accuracy_score(y_test, y_pred)
            f1 = f1_score(y_test, y_pred, average='weighted')
            
            # 메트릭 로깅
            mlflow.log_metric("accuracy", accuracy)
            mlflow.log_metric("f1_score", f1)
            
            # 모델 저장
            mlflow.sklearn.log_model(model, "model")
            
            tuning_results.append({
                'n_estimators': n_est,
                'max_depth': max_d,
                'accuracy': accuracy,
                'f1_score': f1
            })

# 결과 시각화
tuning_df = pd.DataFrame(tuning_results)
print("\n하이퍼파라미터 튜닝 결과:")
print(tuning_df.sort_values('accuracy', ascending=False))

In [ ]:
# 최적 파라미터 찾기
best_params = tuning_df.loc[tuning_df['accuracy'].idxmax()]
print("\n최적 하이퍼파라미터:")
print(best_params)

## 6. 아티팩트 로깅 (차트, 데이터 등)

In [ ]:
with mlflow.start_run(run_name="rf-with-artifacts"):
    # 모델 학습
    model = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)
    model.fit(X_train_scaled, y_train)
    
    # 예측 및 평가
    y_pred = model.predict(X_test_scaled)
    accuracy = accuracy_score(y_test, y_pred)
    
    # 메트릭 로깅
    mlflow.log_metric("accuracy", accuracy)
    
    # 특성 중요도 차트 생성
    feature_importance = pd.DataFrame({
        'feature': wine.feature_names,
        'importance': model.feature_importances_
    }).sort_values('importance', ascending=False)
    
    plt.figure(figsize=(10, 6))
    plt.barh(feature_importance['feature'][:10], feature_importance['importance'][:10])
    plt.xlabel('Importance')
    plt.title('Top 10 Feature Importance')
    plt.gca().invert_yaxis()
    plt.tight_layout()
    plt.savefig('feature_importance.png')
    plt.close()
    
    # 차트를 아티팩트로 로깅
    mlflow.log_artifact('feature_importance.png')
    
    # 모델 저장
    mlflow.sklearn.log_model(model, "model")
    
    print(f"아티팩트가 로깅되었습니다. Accuracy: {accuracy:.4f}")

## 7. 저장된 모델 로드 및 사용

In [ ]:
# 실험에서 최고 성능의 run ID 찾기
experiment = mlflow.get_experiment_by_name(experiment_name)
runs = mlflow.search_runs(
    experiment_ids=[experiment.experiment_id],
    order_by=["metrics.accuracy DESC"],
    max_results=1
)

if len(runs) > 0:
    best_run_id = runs.iloc[0]['run_id']
    best_accuracy = runs.iloc[0]['metrics.accuracy']
    
    print(f"최고 성능 Run ID: {best_run_id}")
    print(f"최고 Accuracy: {best_accuracy:.4f}")
    
    # 모델 로드
    model_uri = f"runs:/{best_run_id}/model"
    loaded_model = mlflow.sklearn.load_model(model_uri)
    
    # 예측
    predictions = loaded_model.predict(X_test_scaled[:5])
    print("\n로드된 모델의 예측:")
    for i, pred in enumerate(predictions):
        print(f"샘플 {i+1}: {wine.target_names[pred]}")

## 8. 실험 결과 조회

In [ ]:
# 모든 실험 결과 조회
all_runs = mlflow.search_runs(
    experiment_ids=[experiment.experiment_id],
    order_by=["metrics.accuracy DESC"]
)

# 주요 컬럼만 선택
columns_to_show = ['run_id', 'metrics.accuracy', 'metrics.f1_score', 
                   'params.model_type', 'params.n_estimators', 'params.max_depth']
display_columns = [col for col in columns_to_show if col in all_runs.columns]

print("\n전체 실험 결과 (정확도 순):")
print(all_runs[display_columns].head(10))

## 요약

이 노트북에서는 MLflow를 사용하여:
1. ✅ 실험 추적 환경 설정
2. ✅ 단일 모델 실험 실행
3. ✅ 여러 모델 비교 실험
4. ✅ 하이퍼파라미터 튜닝 실험
5. ✅ 아티팩트 (차트, 파일) 로깅
6. ✅ 저장된 모델 로드 및 사용
7. ✅ 실험 결과 조회 및 비교

다음 단계: 모델 버전 관리 및 레지스트리 사용